In [21]:
from api.db import DB

GET_SCADAS = """--sql
WITH
    capacity_scadas AS (
        SELECT DISTINCT
            source_key
        FROM sdm_dba.data_catalog
        WHERE
            metric_nice_name ~ 'Tank[0-9]+Capacity'
    )
SELECT source_key, division_id
FROM sdm_dba.data_catalog
WHERE
    metric_nice_name ~ 'Tank[0-9]+Level'
    AND source_key NOT IN(
        SELECT source_key
        FROM capacity_scadas
    );
"""

res = await DB.fetch(GET_SCADAS)
if res is None:
    raise ValueError("No SCADAs found")
df = res
df

source_key,division_id
str,i64
"""6941901""",5
"""6941902""",5
"""6941903""",5
"""6941903""",5
"""69419W1""",5
…,…
"""98750W2""",12
"""110009W2""",26
"""9875001""",12


In [22]:
row_count = df.height
row_count

21

In [23]:
import random

import polars as pl

pl.Config.set_tbl_rows(50)

lower_range = 400
upper_range = 1000
random_tank_sizes = map(
    lambda _: random.uniform(lower_range, upper_range), range(row_count)
)
random_tank_sizes = pl.Series(
    name="tanksize", values=random_tank_sizes, dtype=pl.Float64
)
random_tank_sizes

tanksize
f64
694.857471
789.174369
467.055232
470.865867
574.152953
…
753.741052
778.986218
414.215531


In [24]:
df = df.with_columns(random_tank_sizes)
df

source_key,division_id,tanksize
str,i64,f64
"""6941901""",5,694.857471
"""6941902""",5,789.174369
"""6941903""",5,467.055232
"""6941903""",5,470.865867
"""69419W1""",5,574.152953
…,…,…
"""98750W2""",12,753.741052
"""110009W2""",26,778.986218
"""9875001""",12,414.215531


In [25]:
df = df.rename({"source_key": "scadaid"})
df

scadaid,division_id,tanksize
str,i64,f64
"""6941901""",5,694.857471
"""6941902""",5,789.174369
"""6941903""",5,467.055232
"""6941903""",5,470.865867
"""69419W1""",5,574.152953
…,…,…
"""98750W2""",12,753.741052
"""110009W2""",26,778.986218
"""9875001""",12,414.215531


In [26]:
res = await DB.insert(df, "tank.tank_metadata", return_cols={"scadaid", "division_id"})
res

scadaid,division_id
str,i64
"""6941901""",5
"""6941902""",5
"""6941903""",5
"""69419W1""",5
"""69419W2""",5
…,…
"""98750W2""",12
"""110009W2""",26
"""9875001""",12
